In [1]:
import pandas as pd
import xlrd
import numpy as np
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [17]:
# read file and explore dataset
attacks = pd.read_csv(r'C:\Users\Vanessa\Downloads\Dados\Programas\Repositório Git\attacks.csv', encoding = 'latin-1')
attacks.info()
attacks.head()
attacks.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25723 entries, 0 to 25722
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Case Number             8702 non-null   object 
 1   Date                    6302 non-null   object 
 2   Year                    6300 non-null   float64
 3   Type                    6298 non-null   object 
 4   Country                 6252 non-null   object 
 5   Area                    5847 non-null   object 
 6   Location                5762 non-null   object 
 7   Activity                5758 non-null   object 
 8   Name                    6092 non-null   object 
 9   Sex                     5737 non-null   object 
 10  Age                     3471 non-null   object 
 11  Injury                  6274 non-null   object 
 12  Fatal (Y/N)             5763 non-null   object 
 13  Time                    2948 non-null   object 
 14  Species                 3464 non-null 

(25723, 24)

In [3]:
# clean columns' names
attacks.columns = attacks.columns.str.lower().str.strip()
attacks.columns

Index(['case number', 'date', 'year', 'type', 'country', 'area', 'location',
       'activity', 'name', 'sex', 'age', 'injury', 'fatal (y/n)', 'time',
       'species', 'investigator or source', 'pdf', 'href formula', 'href',
       'case number.1', 'case number.2', 'original order', 'unnamed: 22',
       'unnamed: 23'],
      dtype='object')

In [4]:
# drop irrelevant columns
clean_attacks = attacks.drop(['unnamed: 22', 'unnamed: 23'], axis = 1)

In [18]:
# drop lines if more than 22 columns are null
allnull = attacks.isna().sum(axis = 1) >= 22
notnull = ~allnull
clean_attacks = attacks[notnull]
clean_attacks.shape

(6302, 24)

In [6]:
# clean 'fatal' column
import re
clean_attacks.rename(columns = {'fatal (y/n)':'fatal'}, inplace = True)
clean_attacks['fatal'] = clean_attacks['fatal'].str.lower().str.strip().replace('2017','u', regex = True).replace('m','u', regex = True).replace('unknown','u', regex = True)
clean_attacks['fatal'].fillna('u', inplace = True)

C:\Users\Vanessa\AppData\Local\Temp\ipykernel_12076\3256031532.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_attacks.rename(columns = {'fatal (y/n)':'fatal'}, inplace = True)
C:\Users\Vanessa\AppData\Local\Temp\ipykernel_12076\3256031532.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_attacks['fatal'] = clean_attacks['fatal'].str.lower().str.strip().replace('2017','u', regex = True).replace('m','u', regex = True).replace('unknown','u', regex = True)
C:\Users\Vanessa\AppData\Local\Temp\ipykernel_12076\3256031532.py:5: SettingWithCopyWarning: 

In [7]:
# clean 'sex' column
clean_attacks2 = clean_attacks['sex'].fillna('u', inplace = True)
clean_attacks2 = clean_attacks['sex'].str.lower().str.strip().replace('.', 'u').replace('lli', 'u').replace('n', 'u')
clean_attacks['sex'] = clean_attacks2

C:\Users\Vanessa\AppData\Local\Temp\ipykernel_12076\1072108464.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_attacks2 = clean_attacks['sex'].fillna('u', inplace = True)
C:\Users\Vanessa\AppData\Local\Temp\ipykernel_12076\1072108464.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_attacks['sex'] = clean_attacks2


In [8]:
# clean 'age' column
# these exclusions where less than 1% of DF (23/6302): adult = 2, teens = 16, elderly = 1, young = 3, middle-age = 1 
import re
clean_attacks['age'] = clean_attacks['age'].str.lower().str.strip().fillna('u')
clean_attacks['age'] = clean_attacks['age'].str.replace(r'x|a.m.|f|make line green|teens|teen|"young"|young|adult|\(adult\)|elderly|"middle-age"|>50', 'u')
clean_attacks['age'] = clean_attacks['age'].str.replace('20s','25').str.replace('30s','35').str.replace('40s','45').str.replace('50s','55').str.replace("60's","65").str.replace("60s","65").str.replace('mid-25','25').str.replace('mid-35','35')
clean_attacks['age'] = clean_attacks['age'].str.replace('ca.', '').str.replace('both 11', '11 & 11').str.replace('2 to 3 months', '0.3').str.replace('18 months', '1.5').str.replace('9 months', '0.7')

def clean_numbers(x):
    if 'or' in x or 'to' in x:
        n1 = re.findall('^\d+', x)
        n2 = re.findall('\d+$', x)
        lista_n = n1 + n2
        lista_int = list(map(int, lista_n)) 
        y = sum(lista_int)/2
        return(y)
    elif '½' in x:
        z = re.sub('½', '.5', x)
        return(float(z))
    else:
        return(x)

clean_attacks['age'] = clean_attacks['age'].apply(clean_numbers) 

def clean_and(x):
    if type(x) == str and '&' in x:
        return re.split('[&,]', x)
    else:
        return [x]
        
clean_attacks['age'] = clean_attacks['age'].apply(clean_and) 

new_df = clean_attacks.explode('age')
new_df['age'] = new_df['age'].str.strip().str.replace('?', '')
new_df['age'] = new_df['age'].apply(lambda x: np.nan if x == 'u' or x == '' else float(x))

C:\Users\Vanessa\AppData\Local\Temp\ipykernel_12076\2502510549.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_attacks['age'] = clean_attacks['age'].str.lower().str.strip().fillna('u')
C:\Users\Vanessa\AppData\Local\Temp\ipykernel_12076\2502510549.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_attacks['age'] = clean_attacks['age'].str.replace(r'x|a.m.|f|make line green|teens|teen|"young"|young|adult|\(adult\)|elderly|"middle-age"|>50', 'u')
C:\Users\Vanessa\AppData\Local\Temp\ipykernel_12076\2502510549.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

count    3446.000000
mean       27.410186
std        13.894617
min         0.300000
25%        17.000000
50%        24.000000
75%        35.000000
max        87.000000
Name: age, dtype: float64

In [9]:
# create new column with 'age groups'
ages = ['0-9', '10-19', '20-29', '30-39', '40-49','50-59', '60-69', '70-79', '80-87']  
new_df.loc[:,'age_group'] = pd.cut(new_df['age'], [0, 9, 19, 29, 39, 49, 59, 69, 79, 87], labels = ages, include_lowest= True)

In [10]:
# create data frames with percentages from the clean columns

new_df['fatal'] = new_df['fatal'].apply(lambda x: np.nan if x == 'u' else (x))  
fatal_df = pd.DataFrame(new_df['fatal'].value_counts())
fatal_df.loc[:,'t%'] = fatal_df.apply(lambda x: round((100 * x)/5709, 2), axis=0)
fatal_df

# 5709/6332 not null (90.16%)
# conclusion: most attacks are not fatal

,fatal,t%
n,4316,75.6
y,1393,24.4


In [11]:
new_df['sex'] = new_df['sex'].apply(lambda x: np.nan if x == 'u' else (x)) 
sex_df = pd.DataFrame(new_df['sex'].value_counts())
sex_df.loc[:,'t%'] = sex_df.apply(lambda x: round((100 * x)/5759, 2), axis=0)
sex_df

# 5759/6332 not null (90.95%)
# conclusion: most attacks involve male victims

,sex,t%
m,5120,88.9
f,639,11.1


In [12]:
age_group_df = pd.DataFrame(new_df['age_group'].value_counts())
age_group_df.loc[:,'t%'] = age_group_df.apply(lambda x: round((100 * x)/3446, 2), axis=0)
age_group_df

# 3446/6332 not null (54,4%)
# conclusion: the attacks frequency decreases with age increase, except for young children (0-9)

,age_group,t%
10-19,1093,31.72
20-29,1017,29.51
30-39,560,16.25
40-49,355,10.30
50-59,196,5.69
0-9,118,3.42
60-69,80,2.32
70-79,22,0.64
80-87,5,0.15


In [29]:
new_df['age'].describe()

# conclusion: the mean age of the attacks was 27.4 years old

count    3446.000000
mean       27.410186
std        13.894617
min         0.300000
25%        17.000000
50%        24.000000
75%        35.000000
max        87.000000
Name: age, dtype: float64

In [13]:
fatal_sex = new_df[['fatal', 'sex']].groupby('fatal').value_counts()
fatal_sex_df = pd.DataFrame(fatal_sex) 
fatal_sex_df.loc[:,'t%'] = fatal_sex_df.apply(lambda x: round((100 * x)/5245, 2), axis=0)
fatal_sex_df.columns = ['cases', 't%']
fatal_sex_df.reset_index(inplace = True)
fatal_sex_df.loc[:,'r%'] = fatal_sex_df.apply(lambda x: round((100*x['cases'])/3961, 2) if x['fatal'] == 'n' else round((100*x['cases'])/1284, 2), axis = 1)
fatal_sex_df

# 5245/6332 not null (82,83%)
# conclusions: most fatal cases happen among males

,fatal,sex,cases,t%,r%
0,n,m,3491,66.56,88.13
1,n,f,470,8.96,11.87
2,y,m,1177,22.44,91.67
3,y,f,107,2.04,8.33


In [14]:
fatal_age = new_df[['fatal', 'age_group']].groupby('fatal').value_counts()
fatal_age_df = pd.DataFrame(fatal_age) 
fatal_age_df.loc[:,'t%'] = fatal_age_df.apply(lambda x: round((100 * x)/3201, 2), axis=0)
fatal_age_df.columns = ['cases', 't%']
fatal_age_df.reset_index(inplace = True)
fatal_age_df.loc[:,'r%'] = fatal_age_df.apply(lambda x: round((100*x['cases'])/2594, 2) if x['fatal'] == 'n' else round((100*x['cases'])/607, 2), axis = 1)
fatal_age_df 

# 3201/6332 not null (50,55%)
# conclusions: over 65% of fatal cases happen among ages 10-29

,fatal,age_group,cases,t%,r%
0,n,10-19,834,26.05,32.15
1,n,20-29,751,23.46,28.95
2,n,30-39,425,13.28,16.38
3,n,40-49,277,8.65,10.68
4,n,50-59,151,4.72,5.82
5,n,0-9,87,2.72,3.35
6,n,60-69,52,1.62,2.00
7,n,70-79,13,0.41,0.50
8,n,80-87,4,0.12,0.15
9,y,10-19,198,6.19,32.62


In [15]:
fatal_country = new_df[['fatal', 'country']].groupby('fatal').value_counts()
fatal_country

# conclusions: top 10 countries for fatal attacks are Autralia, USA, South Africa, Papua New Guinea, Mexico, Brazil, Philippines, Reunion, New Zealand and Cuba

fatal  country                              
n      USA                                      1839
       AUSTRALIA                                 926
       SOUTH AFRICA                              408
       BAHAMAS                                    94
       NEW ZEALAND                                91
       PAPUA NEW GUINEA                           74
       BRAZIL                                     64
       ITALY                                      41
       FIJI                                       39
       MEXICO                                     35
       NEW CALEDONIA                              34
       REUNION                                    31
       SPAIN                                      26
       PHILIPPINES                                22
       EGYPT                                      21
       MOZAMBIQUE                                 21
       FRENCH POLYNESIA                           21
       CUBA                                       17
 

In [16]:
fatal_year = new_df[['fatal', 'year']].groupby('fatal').value_counts()
fatal_year 

# conclusions: top 10 years for fatal attacks since 1960 are 2000, 1944, 1993, 1942, 1964, 1962, 1966, 1963, 1954, 1960

fatal  year  
n      2015.0    121
       2017.0    115
       2016.0    113
       2014.0    111
       2011.0    104
       2013.0    102
       2008.0    100
       2012.0     98
       2009.0     97
       2007.0     96
       2010.0     89
       2006.0     86
       2005.0     84
       2001.0     82
       2003.0     79
       2002.0     74
       2000.0     73
       2004.0     69
       1960.0     67
       0.0        64
       1962.0     61
       1961.0     60
       1959.0     58
       1999.0     58
       1998.0     54
       1995.0     53
       2018.0     51
       1996.0     45
       1997.0     44
       1988.0     44
       1994.0     43
       1992.0     41
       1964.0     41
       1963.0     40
       1993.0     40
       1989.0     39
       1983.0     39
       1981.0     38
       1975.0     38
       1966.0     34
       1965.0     34
       1968.0     33
       1990.0     32
       1986.0     32
       1991.0     31
       1970.0     30
       1976.0     30